# 簡単なモデルをつくってみよう

- 前のノートブックのつづき

## E-Cell3を使うための準備

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

## %install_ext https://raw.githubusercontent.com/naito/ecell3-ipython/master/extensions/ecell3.py
## %load_ext ecell3

In [ ]:
import biosim_course

# モデル１

![モデル１概要](./model1.png)

# このモデルに含まれる `Entity`

## `System`
- ルートシステム（細胞外の空間） FullID = `System::/'
- 細胞 FullID = `System:/:Cell'

## `Variable`
- 物質１ (`S1`)　 FullID = `Varible:/Cell:S1'
- 物質２ (`S2`)　 FullID = `Varible:/Cell:S2'

## `Process`
- 物質１ (`E1`)　 FullID = `Process:/Cell:E1'
- 物質２ (`E2`)　 FullID = `Process:/Cell:E2'

# モデルの記法（EM）

## 基本となる書式

![EM書式](./EM1.png)

## クラス名
- `Entity` 名が『生物種名』（イヌ、ネコ etc.）だとしたら、**クラス名** は品種（トイプードル、アメリカンショートヘア etc.）にあたる。
- `System`, `Variable` のクラスは、それぞれ、`System`, `Variable` の１種類だけ。
  - `Variable Variable` と並んでいるが、最初の `Variable` は `Entity` 名、２番目の `Variable` はクラス名を表している。
- `Process`, `Stepper` には多彩なクラスがある。
  - 反応の種類や、時間発展の計算方法によって使い分ける。

## 要素名
- 個々の要素の名前。たとえ話をつづけるなら、個体名（ポチ、ミケ etc.）にあたる。
- FullID の３番目の要素となる。
- あるモデルの中に、同じ FullID をもつ要素はつくれない。
  - したがって、同一の `System`  内に、同じ要素名を持つ `Variable` をつくることはできない。
  - 同様に、同一の `System`  内に、同じ要素名を持つ `Process` をつくることはできない。
  - 同一の `System`  内に、同じ要素名を持つ `Variable` と `Process` をつくることはできる。（`Entity` が異なれば、異なる FullID になるため）


![EM書式](./EM2.png)

## 記法
- **`Entity`名**、**クラス名**を空白文字（スペース）で区切って並べ、次に**要素名**を丸括弧（パーレン）でくくって並べる（空白文字はあってもなくてもよい）。
- つづいて、波括弧（ブレース）でくくった中に、**属性（Property）**とその**値**を列挙する。
- 属性と値は、１件ずつ、末尾にセミコロン（;）を付す。
  - 属性と値の間は空白文字で区切る。
- 属性と値を１つも記述しない場合にも、空っぽの波括弧 `{}` を要素名のあとに記す。
- **改行や空白文字の数に意味はない。** 人間が読みやすいように整えていい。

![EM書式](./EM3.png)

### 複雑な値をもつ属性
- 一部の属性は複数の値の集合のような複雑な値をもつ。
- その場合にも、基本的な書式は変わらない。属性名のあとに空白文字があり、値の後にセミコロンがあるので、その間に記述されている内容は、どんなに長大で複雑であっても、その属性の値である。
  - 上の例では、２行にわたって書かれた文字列が、属性 `VaribleReferenceList` の値である。

# モデルをつくる（１）〜（３）は省略

# （４）`Process` を定義する
- `Variable` と同様、 `Process` も、所属する `System` の属性を書き込む波括弧の中に列挙する。
 
- このモデルでは、**質量作用則（mass action rule）** を表現する `MassActionFluxProcess` クラスを使うことにする。

## 質量作用則とは
- 基質の濃度に応じて反応速度が決まる。
  - 基質濃度が２倍になれば、反応速度が２倍になる、という単純な関係。
  - ２種類の基質が反応を起こす場合、どちらかの濃度が２倍になれば、反応速度は２倍になり、両者の濃度が２倍になければ反応速度は４倍になる。
- 基質が１種類の場合
  - 反応速度 ＝ 単位時間あたり、分子１つあたり、反応が起こる確率
- 基質が２種類以上の場合
  - 反応液の中で基質がランダムに出遭う（衝突する）確率　×　衝突した場合に反応が起こる確率

- 実世界で起こっている物質間相互作用の多くは、ミクロの視点で見ると質量作用則にのっとっている場合が多い。

- 質量作用則では、反応の生成物の影響は考慮しない。
  - 生命システムでは、生成物の量がネガティヴフィードバックを介して影響する場合がしばしば見られる。

### 質量作用則の反応速度をあらわす式
- $n$ 個の基質が反応し、それぞれの濃度が $C_1, C_2,... C_n$ のとき、反応速度 $v$ は下式のように書ける。

　　　　$v = k \times C_1 \times C_2 \times ... \times C_n$

  - ただし、$k$ は速度定数。（反応が起こる確率などをすべて掛けあわせた定数）

## `MassActionFluxProcess` 
- E-Cell3が標準で用意している質量作用則を計算するための `Process` のクラス

### 設定しなければならない属性

#### `k`
- 速度定数

#### `VariableReferenceList`
- この `Process` によって表現する反応に参加する `Variable` と、その参加形式をリスト化したもの。
- `MassActionFluxProcess` に限らず、すべての `Process` クラスは、`VariableReferenceList` 属性を必要とする。

## `VariableReferenceList` の書き方
- `VariableReference` を角括弧でくくって並べる
  - 改行などの必要はないが、読みやすく整えるとよい。


![EM書式](./EM4.png)

## `VariableReference` の書き方
- ひとつの `VariableReference` には、**参照名**、**Variable の FullID**、**Coefficient** を**空白文字**で区切って列挙する。

### 参照名
- `Process` 内での呼び名。自由につけて良い。

### `Variable` の `FullID`

### Coefficient
- 化学量論係数
- この反応による物質（`Variable`）の増減。
  - このモデルでは、`Process E1`では１分子の`S1`が消費され、１分子の`S2`が生じる。
  - `Process E2`では１分子の`S2`が消費され、１分子の`S1`が生じる。


　　　　$\require{AMScd} \begin{CD} S_1 @>{E_1}>> S_2 \end{CD}$

　　　　$\require{AMScd} \begin{CD} S_2 @>{E_2}>> S_1 \end{CD}$

- **整数**で記述する。
- 順不同

## `MassActionFluxProcess` による `VaruableReferenceList` の解釈
- Coefficient が負の値の`Variable`を**基質**として扱う。
- Coefficient が正の値の`Variable`を**生成物**として扱う。
- 反応速度 $v$ は下式で計算される。

　　　　$v = k \times C_1^{h_1} \times C_2^{h_2} \times ... \times C_n^{h_n}$

  - ただし、$h_n$ は $n$ 番目の基質の Coefficient（化学量論係数）。


## EM への記述
- 速度定数 `k` は、`Process` ごとに自由に設定して良い。大きい値にするほど反応速度が大きくなる。

In [ ]:
EM = '''
Stepper ODEStepper( ODE ){}

System System( / )
{
    StepperID    ODE;

    Variable Variable( SIZE )
    {
        Value    1;
    }
}

System System( /Cell )
{
    StepperID    ODE;

Variable Variable( SIZE )
    {
        Value    1;
    }

    Variable Variable( S1 )
    {
        Value    10000;
    }

    Variable Variable( S2 )
    {
        Value    10000;
    }

    Process MassActionFluxProcess( E1 )
    {
        k    0.1;
        VariableReferenceList
            [ S1  Variable:/Cell:S1 -1 ]
            [ S2  Variable:/Cell:S2  1 ];
    }

    Process MassActionFluxProcess( E2 )
    {
        k    0.2;
        VariableReferenceList
            [ S2  Variable:/Cell:S2 -1 ]
            [ S1  Variable:/Cell:S1  1 ];
    }

}
'''

# モデルを走らせてみる

In [ ]:
setModel(EM, "model")

print 't = {}'.format( getCurrentTime() )

## Loggerをつくる

In [ ]:
target_SystemPath_list = ( '/', '/Cell', )
Target_Properties = { 'Variable': ['Value', 'Velocity'], }
Logger_dict = {}

for target_SystemPath in target_SystemPath_list:
    for E_type, Properties in Target_Properties.items():
        for E in getEntityList( E_type, target_SystemPath ):
            for p in Properties:
                FullPN = ':'.join( ( E_type, target_SystemPath, E, p ) )
                Logger_dict[ FullPN ] = createLoggerStub( FullPN )
                Logger_dict[ FullPN ].create()

## シミュレーションを実行する

In [ ]:
step_width = 60.0  # min

run( step_width )
print 't = {}'.format( getCurrentTime() )

## グラフを描いてみる

In [ ]:
Data_dict = {}
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 0, getCurrentTime(), getCurrentTime() / 100 ) )[ :, :2 ]

FullPNs_for_plot = [ 
    'Variable:/Cell:S1:Value',
    'Variable:/Cell:S2:Value',
    ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

## 速度定数 `k` を変えるとどうなるか？
- 現在の `k` の値を確認。

In [ ]:
E1_Stub = createEntityStub( 'Process:/Cell:E1' )
E2_Stub = createEntityStub( 'Process:/Cell:E2' )

print( 'k_E1 = {}'.format( E1_Stub['k'] ))
print( 'k_E2 = {}'.format( E2_Stub['k'] ))


- `Process E2` の `k` を変えてみる。

In [ ]:
E2_Stub['k'] = 0.3

print( 'k_E2 = {}'.format( E2_Stub['k'] ))

In [ ]:
run( step_width )
print 't = {}'.format( getCurrentTime() )

In [ ]:
Data_dict = {}
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 0, getCurrentTime(), getCurrentTime() / 100 ) )[ :, :2 ]

FullPNs_for_plot = [ 
    'Variable:/Cell:S1:Value',
    'Variable:/Cell:S2:Value',
    ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

## 考察
- 速度定数 `k` と定常状態の関係を考えてみよう。